# Multithreading
Tutorial by Jonas Wilfert, Tobias Thummerer

🚧 This tutorial is under revision and will be replaced by an up-to-date version soon 🚧

## License

In [1]:
# Copyright (c) 2021 Tobias Thummerer, Lars Mikelsons, Josef Kircher, Johannes Stoljar, Jonas Wilfert
# Licensed under the MIT license. 
# See LICENSE (https://github.com/thummeto/FMI.jl/blob/main/LICENSE) file in the project root for details.

## Motivation
This Julia Package *FMI.jl* is motivated by the use of simulation models in Julia. Here the FMI specification is implemented. FMI (*Functional Mock-up Interface*) is a free standard ([fmi-standard.org](https://fmi-standard.org/)) that defines a container and an interface to exchange dynamic models using a combination of XML files, binaries and C code zipped into a single file. The user can thus use simulation models in the form of an FMU (*Functional Mock-up Units*). Besides loading the FMU, the user can also set values for parameters and states and simulate the FMU both as co-simulation and model exchange simulation.

## Introduction to the example
This example shows how to parallelize the computation of an FMU in FMI.jl. We can compute a batch of FMU-evaluations in parallel with different initial settings.
Parallelization can be achieved using multithreading or using multiprocessing. This example shows **multithreading**, check `multiprocessing.ipynb` for multiprocessing.
Advantage of multithreading is a lower communication overhead as well as lower RAM usage.
However in some cases multiprocessing can be faster as the garbage collector is not shared.


The model used is a one-dimensional spring pendulum with friction. The object-orientated structure of the *SpringFrictionPendulum1D* can be seen in the following graphic.

![svg](https://github.com/thummeto/FMI.jl/blob/main/docs/src/examples/pics/SpringFrictionPendulum1D.svg?raw=true)  


## Target group
The example is primarily intended for users who work in the field of simulations. The example wants to show how simple it is to use FMUs in Julia.


## Other formats
Besides, this [Jupyter Notebook](https://github.com/thummeto/FMI.jl/blob/examples/examples/src/multithreading.ipynb) there is also a [Julia file](https://github.com/thummeto/FMI.jl/blob/examples/examples/src/multithreading.jl) with the same name, which contains only the code cells and for the documentation there is a [Markdown file](https://github.com/thummeto/FMI.jl/blob/examples/examples/src/multithreading.md) corresponding to the notebook.  


## Getting started

### Installation prerequisites
|     | Description                       | Command                   | Alternative                                    |   
|:----|:----------------------------------|:--------------------------|:-----------------------------------------------|
| 1.  | Enter Package Manager via         | ]                         |                                                |
| 2.  | Install FMI via                   | add FMI                   | add " https://github.com/ThummeTo/FMI.jl "     |
| 3.  | Install FMIZoo via                | add FMIZoo                | add " https://github.com/ThummeTo/FMIZoo.jl "  |
| 4.  | Install FMICore via               | add FMICore               | add " https://github.com/ThummeTo/FMICore.jl " |
| 5.  | Install Folds via                 | add Folds                 |                                                |
| 6.  | Install BenchmarkTools via        | add BenchmarkTools        |                                                |

## Code section

To run the example, the previously installed packages must be included. 

In [2]:
# imports
using FMI
using FMIZoo
using Folds
using BenchmarkTools
using DifferentialEquations

┌ Warning: Circular dependency detected. Precompilation will be skipped for:
│   NonlinearSolveBaseSparseMatrixColoringsExt [e3ecd195-ca82-5397-9546-f380c1e34951]
│   DiffEqBaseChainRulesCoreExt [b00db79b-61e3-50fb-b26f-2d35b2d9e4ed]
│   Transducers [28d57a85-8fef-5791-bfe6-a80928e7c999]
│   NonlinearSolve [8913a72c-1f9b-4ce2-8d82-65094dcecaec]
│   OrdinaryDiffEq [1dea7af3-3e70-54e6-95c3-0bf5283fa5ed]
│   DifferentialEquationsFMIExt [232470a1-1d28-551b-8e3b-d6141e70703a]
│   NonlinearSolveBaseForwardDiffExt [63d416d0-6995-5965-81e0-55251226d976]
│   Folds [41a02a25-b8f0-4f67-bc48-60067656b558]
│   LineSearchLineSearchesExt [8d20b31a-8b56-511a-b573-0bef60e8c8c7]
│   NonlinearSolveBandedMatricesExt [8800daa3-e725-5fa8-982f-091420a833d6]
│   OrdinaryDiffEqFunctionMap [d3585ca7-f5d3-4ba6-8057-292ed1abd90f]
│   LinearSolveEnzymeExt [133222a9-3015-5ee0-8b28-65fc8ed13c28]
│   OrdinaryDiffEqLinear [521117fe-8c41-49f8-b3b6-30780b3f0fb5]
│   OrdinaryDiffEqTsit5 [b1df2697-797e-41e3-8120-5422d3b24

[ Info: Precompiling FMI [14a09403-18e3-468f-ad8a-74f8dda2d9ac]


┌ Warning: Module DatesExt with build ID ffffffff-ffff-ffff-0000-00f398717ce5 is missing from the cache.
│ This may mean DatesExt [0361c7f5-3687-5641-8bd2-a1de0c64d1ed] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:2011


┌ Error: Error during loading of extension DatesExt of Accessors, use `Base.retry_load_extensions()` to retry.
│   exception =
│    1-element ExceptionStack:
│    Declaring __precompile__(false) is not allowed in files that are being precompiled.
│    Stacktrace:
│      [1] _require(pkg::Base.PkgId, env::Nothing)
│        @ Base .\loading.jl:2015
│      [2] __require_prelocked(uuidkey::Base.PkgId, env::Nothing)
│        @ Base .\loading.jl:1875
│      [3] #invoke_in_world#3
│        @ .\essentials.jl:926 [inlined]
│      [4] invoke_in_world
│        @ .\essentials.jl:923 [inlined]
│      [5] _require_prelocked
│        @ .\loading.jl:1866 [inlined]
│      [6] _require_prelocked
│        @ .\loading.jl:1865 [inlined]
│      [7] run_extension_callbacks(extid::Base.ExtensionId)
│        @ Base .\loading.jl:1358
│      [8] run_extension_callbacks(pkgid::Base.PkgId)
│        @ Base .\loading.jl:1393
│      [9] run_package_callbacks(modkey::Base.PkgId)
│        @ Base .\loading.jl:1218
│    

┌ Warning: Circular dependency detected. Precompilation will be skipped for:
│   NonlinearSolveBaseSparseMatrixColoringsExt [e3ecd195-ca82-5397-9546-f380c1e34951]
│   DiffEqBaseChainRulesCoreExt [b00db79b-61e3-50fb-b26f-2d35b2d9e4ed]
│   Transducers [28d57a85-8fef-5791-bfe6-a80928e7c999]
│   NonlinearSolve [8913a72c-1f9b-4ce2-8d82-65094dcecaec]
│   OrdinaryDiffEq [1dea7af3-3e70-54e6-95c3-0bf5283fa5ed]
│   DifferentialEquationsFMIExt [232470a1-1d28-551b-8e3b-d6141e70703a]
│   NonlinearSolveBaseForwardDiffExt [63d416d0-6995-5965-81e0-55251226d976]
│   Folds [41a02a25-b8f0-4f67-bc48-60067656b558]
│   LineSearchLineSearchesExt [8d20b31a-8b56-511a-b573-0bef60e8c8c7]
│   NonlinearSolveBandedMatricesExt [8800daa3-e725-5fa8-982f-091420a833d6]
│   OrdinaryDiffEqFunctionMap [d3585ca7-f5d3-4ba6-8057-292ed1abd90f]
│   LinearSolveEnzymeExt [133222a9-3015-5ee0-8b28-65fc8ed13c28]
│   OrdinaryDiffEqLinear [521117fe-8c41-49f8-b3b6-30780b3f0fb5]
│   OrdinaryDiffEqTsit5 [b1df2697-797e-41e3-8120-5422d3b24

...

[ Info: Precompiling TransducersLazyArraysExt [cdbecb60-77cf-500a-86c2-8d8bbf22df88]


┌ Warning: Circular dependency detected. Precompilation will be skipped for:
│   NonlinearSolveBaseSparseMatrixColoringsExt [e3ecd195-ca82-5397-9546-f380c1e34951]
│   DiffEqBaseChainRulesCoreExt [b00db79b-61e3-50fb-b26f-2d35b2d9e4ed]
│   Transducers [28d57a85-8fef-5791-bfe6-a80928e7c999]
│   NonlinearSolve [8913a72c-1f9b-4ce2-8d82-65094dcecaec]
│   OrdinaryDiffEq [1dea7af3-3e70-54e6-95c3-0bf5283fa5ed]
│   DifferentialEquationsFMIExt [232470a1-1d28-551b-8e3b-d6141e70703a]
│   NonlinearSolveBaseForwardDiffExt [63d416d0-6995-5965-81e0-55251226d976]
│   Folds [41a02a25-b8f0-4f67-bc48-60067656b558]
│   LineSearchLineSearchesExt [8d20b31a-8b56-511a-b573-0bef60e8c8c7]
│   NonlinearSolveBandedMatricesExt [8800daa3-e725-5fa8-982f-091420a833d6]
│   OrdinaryDiffEqFunctionMap [d3585ca7-f5d3-4ba6-8057-292ed1abd90f]
│   LinearSolveEnzymeExt [133222a9-3015-5ee0-8b28-65fc8ed13c28]
│   OrdinaryDiffEqLinear [521117fe-8c41-49f8-b3b6-30780b3f0fb5]
│   OrdinaryDiffEqTsit5 [b1df2697-797e-41e3-8120-5422d3b24

[ Info: Precompiling DifferentialEquations [0c46a032-eb83-5123-abaf-570d42b7fbaa]


┌ Warning: Circular dependency detected. Precompilation will be skipped for:
│   NonlinearSolveBaseSparseMatrixColoringsExt [e3ecd195-ca82-5397-9546-f380c1e34951]
│   DiffEqBaseChainRulesCoreExt [b00db79b-61e3-50fb-b26f-2d35b2d9e4ed]
│   Transducers [28d57a85-8fef-5791-bfe6-a80928e7c999]
│   NonlinearSolve [8913a72c-1f9b-4ce2-8d82-65094dcecaec]
│   OrdinaryDiffEq [1dea7af3-3e70-54e6-95c3-0bf5283fa5ed]
│   DifferentialEquationsFMIExt [232470a1-1d28-551b-8e3b-d6141e70703a]
│   NonlinearSolveBaseForwardDiffExt [63d416d0-6995-5965-81e0-55251226d976]
│   Folds [41a02a25-b8f0-4f67-bc48-60067656b558]
│   LineSearchLineSearchesExt [8d20b31a-8b56-511a-b573-0bef60e8c8c7]
│   NonlinearSolveBandedMatricesExt [8800daa3-e725-5fa8-982f-091420a833d6]
│   OrdinaryDiffEqFunctionMap [d3585ca7-f5d3-4ba6-8057-292ed1abd90f]
│   LinearSolveEnzymeExt [133222a9-3015-5ee0-8b28-65fc8ed13c28]
│   OrdinaryDiffEqLinear [521117fe-8c41-49f8-b3b6-30780b3f0fb5]
│   OrdinaryDiffEqTsit5 [b1df2697-797e-41e3-8120-5422d3b24

[ Info: Precompiling DifferentialEquationsFMIExt [232470a1-1d28-551b-8e3b-d6141e70703a]


First, check the amount of available threads:

In [3]:
Threads.nthreads()

1

If the number of available threads doesn't match your expections, you can increase the number of threads available to the Julia process like described [here](https://docs.julialang.org/en/v1/manual/multi-threading/#Starting-Julia-with-multiple-threads).

### Simulation setup

Next, the start time and end time of the simulation are set. Here we also decide the size of the batch.

In [4]:
t_start = 0.0
t_step = 0.1
t_stop = 10.0
tspan = (t_start, t_stop)
tData = collect(t_start:t_step:t_stop)

# Best if batchSize is a multiple of the threads/cores
batchSize = Threads.nthreads()

# Define an array of arrays randomly
input_values = collect(collect.(eachrow(rand(batchSize,2))))


1-element Vector{Vector{Float64}}:
 [0.28142814679649286, 0.8081805263090995]

We need to instantiate one FMU for each parallel execution, as they cannot be easily shared among different threads.

In [5]:
# a single FMU to compare the performance
realFMU = loadFMU("SpringPendulum1D", "Dymola", "2022x")

# the FMU batch
realFMUBatch = [loadFMU("SpringPendulum1D", "Dymola", "2022x") for _ in 1:batchSize]

1-element Vector{FMU2}:
 Model name:	SpringPendulum1D
Type:		1

We define a helper function to calculate the FMU solution and combine it into an Matrix.

In [6]:
function runCalcFormatted(fmu::FMU2, x0::Vector{Float64}, recordValues::Vector{String}=["mass.s", "mass.v"])
    data = simulateME(fmu, tspan; recordValues=recordValues, saveat=tData, x0=x0, showProgress=false, dtmax=1e-4)
    return reduce(hcat, data.states.u)
end

runCalcFormatted (generic function with 2 methods)

Running a single evaluation is pretty quick, therefore the speed can be better tested with BenchmarkTools.

In [7]:
@benchmark data = runCalcFormatted(realFMU, rand(2))

BenchmarkTools.Trial: 2 samples with 1 evaluation per sample.
 Range (min … max):  3.083 s …  3.096 s  ┊ GC (min … max): 0.35% … 0.75%
 Time  (median):     3.090 s             ┊ GC (median):    0.55%
 Time  (mean ± σ):   3.090 s ± 9.213 ms  ┊ GC (mean ± σ):  0.55% ± 0.28%

  █                                                      █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  3.08 s        Histogram: frequency by time         3.1 s <

 Memory estimate: 300.75 MiB, allocs estimate: 7602420.

### Single Threaded Batch Execution
To compute a batch we can collect multiple evaluations. In a single threaded context we can use the same FMU for every call.

In [8]:
println("Single Threaded")
@benchmark collect(runCalcFormatted(realFMU, i) for i in input_values)

Single Threaded


BenchmarkTools.Trial: 2 samples with 1 evaluation per sample.
 Range (min … max):  3.075 s …   3.117 s  ┊ GC (min … max): 0.43% … 0.36%
 Time  (median):     3.096 s              ┊ GC (median):    0.39%
 Time  (mean ± σ):   3.096 s ± 29.520 ms  ┊ GC (mean ± σ):  0.39% ± 0.05%

  █                                                       █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  3.08 s         Histogram: frequency by time        3.12 s <

 Memory estimate: 300.75 MiB, allocs estimate: 7602423.

### Multithreaded Batch Execution
In a multithreaded context we have to provide each thread it's own fmu, as they are not thread safe.
To spread the execution of a function to multiple threads, the library `Folds` can be used.

In [9]:
println("Multi Threaded")
@benchmark Folds.collect(runCalcFormatted(fmu, i) for (fmu, i) in zip(realFMUBatch, input_values))

Multi Threaded


BenchmarkTools.Trial: 2 samples with 1 evaluation per sample.
 Range (min … max):  3.063 s …  3.072 s  ┊ GC (min … max): 0.44% … 0.36%
 Time  (median):     3.067 s             ┊ GC (median):    0.40%
 Time  (mean ± σ):   3.067 s ± 5.954 ms  ┊ GC (mean ± σ):  0.40% ± 0.06%

  █                                                      █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  3.06 s        Histogram: frequency by time        3.07 s <

 Memory estimate: 300.75 MiB, allocs estimate: 7602438.

As you can see, there is a significant speed-up in the median execution time. But: The speed-up is often much smaller than `Threads.nthreads()`, this has different reasons. For a rule of thumb, the speed-up should be around `n/2` on a `n`-core-processor with `n` threads for the Julia process.

### Unload FMU

After calculating the data, the FMU is unloaded and all unpacked data on disc is removed.

In [10]:
unloadFMU(realFMU)
unloadFMU.(realFMUBatch)

1-element Vector{Nothing}:
 nothing

### Summary

In this tutorial it is shown how multi threading with `Folds.jl` can be used to improve the performance for calculating a Batch of FMUs.